In [1]:
from flask import Flask, jsonify, request
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
from config import pg_pass



In [2]:
rds_connection_string = (f'postgres:{pg_pass}@localhost:5432/Movies')
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.connect()

In [3]:
engine.table_names()

['imdb_movies_b', 'imdb_movies', 'imdb_ratings', 'imdb_movies_global']

In [4]:
data = pd.read_sql_query('SELECT * FROM imdb_movies_global LIMIT 100', conn)
data.head()


,imdb_title_id,title,year_mv,duration,genre_1,genre_2,genre_3,country_1,country_2,country_3,country_4,country_5,lang_1,lang_2,lang_3,lang_4,lang_5,avg_vote_f
0,tt0000574,The Story of the Kelly Gang,1906,70,Biography,Crime,Drama,Australia,None,None,None,None,None,None,None,None,None,6.1
1,tt0001892,Den sorte drøm,1911,53,Drama,None,None,Germany,Denmark,None,None,None,None,None,None,None,None,5.8
2,tt0002101,Cleopatra,1912,100,Drama,History,None,USA,None,None,None,None,English,None,None,None,None,5.2
3,tt0002452,Independenta Romaniei,1912,120,History,War,None,Romania,None,None,None,None,None,None,None,None,None,6.7
4,tt0002461,Richard III,1912,55,Drama,None,None,France,USA,None,None,None,English,None,None,None,None,5.5


In [5]:
movies_data = data.set_index('imdb_title_id').T.to_dict('dict')
movies_data

{'tt0000574': {'title': 'The Story of the Kelly Gang',
  'year_mv': '1906',
  'duration': 70,
  'genre_1': 'Biography',
  'genre_2': ' Crime',
  'genre_3': ' Drama',
  'country_1': 'Australia',
  'country_2': None,
  'country_3': None,
  'country_4': None,
  'country_5': None,
  'lang_1': 'None',
  'lang_2': None,
  'lang_3': None,
  'lang_4': None,
  'lang_5': None,
  'avg_vote_f': 6.1},
 'tt0001892': {'title': 'Den sorte drøm',
  'year_mv': '1911',
  'duration': 53,
  'genre_1': 'Drama',
  'genre_2': None,
  'genre_3': None,
  'country_1': 'Germany',
  'country_2': ' Denmark',
  'country_3': None,
  'country_4': None,
  'country_5': None,
  'lang_1': None,
  'lang_2': None,
  'lang_3': None,
  'lang_4': None,
  'lang_5': None,
  'avg_vote_f': 5.8},
 'tt0002101': {'title': 'Cleopatra',
  'year_mv': '1912',
  'duration': 100,
  'genre_1': 'Drama',
  'genre_2': ' History',
  'genre_3': None,
  'country_1': 'USA',
  'country_2': None,
  'country_3': None,
  'country_4': None,
  'country_

In [ ]:
#STARTING FLAST SERVER
app = Flask(__name__)

#DEFINING HOME PAGE
@app.route('/')
def home():
    return "Home Page"

#RETURNING ALL MOVIE_DATA IN JASON FORMAT
@app.route('/all_movies')
def all_movies():
    return jsonify(movies_data)


#RETURNING SPECIFIC MOVIE RECORD BASED ON TITLE
@app.route('/movie/<title>')
def id(title):
    titleDF = data.loc[data['title'] == title.capitalize()]
    titleDic = titleDF.set_index('imdb_title_id').T.to_dict('dict')
    try:
        return jsonify(titleDic)
    except:
        return "Movie not available"

#RETURNING ALL MOVIE RECORDS
@app.route('/all_titles')
def all_titles():
    titleDic = data['title'].to_dict()
    try:
        return jsonify(titleDic)
    except:
        return "Movie not available"

#RETURNING ONLY A SPECIFIC YEAR
@app.route('/year/<year>')
def year(year):
    yearDF = data.loc[data['year_mv'] == str(year)]
    yearDic = yearDF.set_index('imdb_title_id').T.to_dict('dict')
    try:
        return jsonify(yearDic)
    except:
        return "Year not found"
    
#RETURNING RECORDS PER GENRE AND LANGUAGE
@app.route('/filter/<genre>/<language>')
def filter(genre=None, language=None):
    filterDF = data.loc[(data['genre_1'] == genre.capitalize()) & (data['lang_1'] == language.capitalize())]
    filterDic = filterDF.set_index('imdb_title_id').T.to_dict('dict')
    try:
        return jsonify(filterDic)
    except:
        return "Record not found"





    
app.run()                                                                         

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/May/2021 12:16:15] "GET /movie/cleopatra HTTP/1.1" 200 -
